In [1]:
from argparse import Namespace
import os
import sys

os.chdir("/playpen-nas-ssd/luchao/projects/SAM/")

import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image
import torchvision.transforms.functional as F
import time
import torch.nn as nn
import copy
import shutil
import collections

# from criteria.aging_loss import AgingLoss
from criteria.aging_loss_fpage import AgingLoss



aging_loss = AgingLoss(opts=None).cuda()
# predict_age = aging_loss.extract_ages(result_tensor.unsqueeze(0)).item()

def iterate_over_img_dir(img_dir):
    IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if any(file.endswith(extension) for extension in IMG_EXTENSIONS):
                yield os.path.join(root, file)

/playpen-nas-ssd/gongbang/miniconda3/envs/arc2face/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/playpen-nas-ssd/luchao/projects/SAM/criteria/aging_loss_fpage.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization

In [2]:
def calculate_age_error(img_dir):
    results_age_pred = collections.defaultdict(list)
    for img_path in iterate_over_img_dir(img_dir):
        img_name = os.path.basename(img_path)
        # if img_name.count('_') == 1:
        #     # print(f'Skip input image: {img_name}')
        #     continue
        if any([keyword in img_name for keyword in ['inversion']]):
            # print(f'Skip inversion image: {img_name}')
            continue
        img = Image.open(img_path).convert('RGB') # (3, 1024, 1024)
        img_tensor = F.to_tensor(img).unsqueeze(0).cuda() # (1, 3, 1024, 1024)
        predicted_age = aging_loss.extract_ages(img_tensor).item()
        target_age = int(img_name.split('.')[0].split('_')[-1])
        assert 0 <= target_age <= 100, f'Invalid age: {target_age}'
        results_age_pred[target_age].append(predicted_age)
    age_errors = []
    for target_age, pred_ages in results_age_pred.items():
        # ----------------------------------- todo ----------------------------------- #
        # if target_age > 70:
        #     # * used for benchmarking against other baselines as some baselines do not support > 70
        #     continue
        age_errors.append(abs(np.mean(pred_ages) - target_age))
    return np.mean(age_errors)
    # return age_errors

In [4]:
# ours = []
# mytm = []
# fading = []
# flowedit = []
# rfsolver = []
# rfinversion = []
# fireflow = []
# flowchef = []
fading_rfsolver = []

In [5]:
celebs = ['al', 'charles', 'chow', 'diego', 'elaine', 'elizabeth', 'jackie', 'jennifer', 'nicole', 'oprah', 'robert', 'thatcher']

for celeb in celebs:
    exp_dir_parent = f'/playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/{celeb}'
    # exp_dir_parent = f'/playpen-nas-ssd/gongbang/personalized/{celeb}'

    for exp in sorted(os.listdir(exp_dir_parent)):
        exp_dir = os.path.join(exp_dir_parent, exp)
        if not os.path.exists(exp_dir):
            print(f'{exp_dir} does not exist')
            continue
        # ! change here
        # if not f'pretrained_{celeb}_70' in exp_dir:
        #     continue
        # if not f'pretrained_{celeb}_40' in exp_dir:
        #     continue
        # if not '2_55' in exp_dir:
        #     continue

        # single_img_dir = os.path.join(exp_dir_parent, exp, 'single_img')
        age_error = calculate_age_error(exp_dir)
        print(f'Experiment: {exp_dir}')
        print(f'Age error: {age_error}')
        print('-'*10)
        fading_rfsolver.append(age_error)

Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/al/alcoholism
Age error: 7.579900122308111
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/al/goodskin
Age error: 5.517092110274674
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/al/hairloss
Age error: 8.578771095771296
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/al/poorskin
Age error: 6.4093018568955475
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/al/weight
Age error: 6.651335431383801
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/charles/alcoholism
Age error: 9.69101708167805
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/charles/goodskin
Age error: 8.119153236856267
----------
Experiment: /playpen-nas-ssd/gongbang/comparisons/rf-solver-fading/charles/hairloss
Age error: 8.95614006405785
----------
Experiment: /playpen-nas-ssd/gon

In [6]:
np.mean(fading_rfsolver)

8.702744971853173